<a href="https://colab.research.google.com/github/claudelepere/ML_GitHub/blob/main/BERT_for_multi_label_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Do not use conda in Colab, do not try to install with conda in Colab
# Do not launch wandb (cannot connect to the W&B server): 3 things: #!pip install, #import, os.environ disable

!pip install -q accelerate
!pip install -q huggingface_hub
!pip install -q transformers datasets
#!pip install -q wandb

import json
import numpy as np
import os
import sys
import time
import torch
#import wandb
os.environ["WANDB_DISABLE_CODE"] = "true"

from datasets              import DatasetDict
from google.colab          import auth, drive, files, userdata
from huggingface_hub       import create_repo, login, upload_file
from huggingface_hub.utils import RepositoryNotFoundError
from sklearn.metrics       import accuracy_score, average_precision_score, f1_score, precision_score, recall_score, roc_auc_score
from transformers          import AutoModelForSequenceClassification, AutoTokenizer, EvalPrediction, Trainer, TrainingArguments


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
"""
# Check the Python version
print(sys.version)
print()

# Get the installed packages (you can see that conda is not installed (do not install it))
!pip list
print()

# Check system information
!cat /etc/os-release
!uname -m
print()

# Check the GPU details (only if the runtime type is T4 GPU)
#!nvidia-smi
#print()

# Check RAM
!free -h
print()

# Check disk space
!df -h
print()

# Get environment variables
for key, value in os.environ.items():
    print(f"{key}: {value}")
"""
import os, torch
print(f"currentdir: {os.getcwd()}")
print()

# Check the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")


currentdir: /content

device: cuda


In [3]:
### Google Colab authenticate

# Required for accessing Colab Secrets
#auth.authenticate_user()


In [4]:
### Hugging Face authenticate

os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")    # Store the key in os.environ
hf_token               = os.environ.get('HF_TOKEN')
login(token=hf_token)

# Verify
!huggingface-cli whoami


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


claudelepere


In [5]:
### Create the skill_classification repo on the Hugging Face Hub

HF_name         = "claudelepere/skill_classification"
repo_id_model   = HF_name
repo_id_dataset = HF_name

repo_model_url = create_repo(
    repo_id   = repo_id_model,
    repo_type = "model",
    private   = True,
    exist_ok  = True
    )
print(f"Repo model url: {repo_model_url} created successfully as a private repo.")

repo_dataset_url = create_repo(
    repo_id   = repo_id_dataset,
    repo_type = "dataset",
    private   = True,
    exist_ok  = True
    )
print(f"Repo datasets url: {repo_dataset_url} created successfully as a private repo.")

repo_id_dataset = f"datasets/{HF_name}"

print(f"repo_id_model: {repo_id_model}")
print(f"repo_id_dataset: {repo_id_dataset}")


Repo model url: https://huggingface.co/claudelepere/skill_classification created successfully as a private repo.
Repo datasets url: https://huggingface.co/datasets/claudelepere/skill_classification created successfully as a private repo.
repo_id_model: claudelepere/skill_classification
repo_id_dataset: datasets/claudelepere/skill_classification


In [6]:

"""
### Weights & Biases (W&B, wandb) authenticate

os.environ["WANDB_API_KEY"] = userdata.get("WANDB_API_KEY")        # Store the key in os.environ
wandb_api_key               = os.environ.get('WANDB_API_KEY')

wandb.login(relogin=True)
wandb.init(project="skill_classification", entity="claudelepere")
"""

'\n### Weights & Biases (W&B, wandb) authenticate\n\nos.environ["WANDB_API_KEY"] = userdata.get("WANDB_API_KEY")        # Store the key in os.environ\nwandb_api_key               = os.environ.get(\'WANDB_API_KEY\')\n\nwandb.login(relogin=True)\nwandb.init(project="skill_classification", entity="claudelepere")\n'

In [7]:
"""
wandb.finish
time.sleep(20)

try:
  wandb.init(
      project="skill_classification",
      entity="claudelepere",
      )
except wandb.errors.CommError as e:
  print(f"CommError: {e}")
except Exception as e:
  print(f"Exception: {e}")
"""

'\nwandb.finish\ntime.sleep(20)\n\ntry:\n  wandb.init(\n      project="skill_classification",\n      entity="claudelepere",\n      )\nexcept wandb.errors.CommError as e:\n  print(f"CommError: {e}")\nexcept Exception as e:\n  print(f"Exception: {e}")\n'

# My fine-tuning BERT (and friends) for multi-label text classification

In this notebook, we are going to fine-tune BERT to predict one or more labels for a given piece of text. Note that this notebook illustrates how to fine-tune a bert-base-uncased model, but you can also fine-tune a RoBERTa, DeBERTa, DistilBERT, CANINE, ... checkpoint in the same way.

All of those work in the same way: they add a **linear layer on top of the base model, which is used to produce a tensor of shape (batch_size, num_labels)**, indicating the unnormalized scores for a number of labels for every example in the batch.



## Set-up environment

First, we install the libraries which we'll use: HuggingFace Transformers and Datasets.

## Load dataset

Next, let's download a multi-label text classification dataset from the [hub](https://huggingface.co/).

At the time of writing, I picked a random one as follows:   

* first, go to the "datasets" tab on huggingface.co
* next, select the "multi-label-classification" tag on the left as well as the the "1k<10k" tag (fo find a relatively small dataset).

Note that you can also easily load your local data (i.e. csv files, txt files, Parquet files, JSON, ...) as explained [here](https://huggingface.co/docs/datasets/loading.html#local-and-remote-files).



In [8]:
### Upload and unzip the dataset zip file

uploaded_files     = files.upload()
uploaded_file_name = list(uploaded_files.keys())[0]
print(f"uploaded_file_name: {uploaded_file_name} {len(uploaded_files)}")

!unzip {uploaded_file_name}

unzipped_file_dir_name = os.path.splitext(uploaded_file_name)[0]
print(f"unzipped_file_dir_name: {unzipped_file_dir_name}")


Saving dataset_8910_1087_68_204.zip to dataset_8910_1087_68_204.zip
uploaded_file_name: dataset_8910_1087_68_204.zip 1
Archive:  dataset_8910_1087_68_204.zip
   creating: dataset_8910_1087_68_204/
   creating: dataset_8910_1087_68_204/test/
   creating: dataset_8910_1087_68_204/train/
   creating: dataset_8910_1087_68_204/validation/
  inflating: dataset_8910_1087_68_204/dataset_dict.json  
  inflating: dataset_8910_1087_68_204/test/data-00000-of-00001.arrow  
  inflating: dataset_8910_1087_68_204/test/dataset_info.json  
  inflating: dataset_8910_1087_68_204/test/state.json  
  inflating: dataset_8910_1087_68_204/train/data-00000-of-00001.arrow  
  inflating: dataset_8910_1087_68_204/train/dataset_info.json  
  inflating: dataset_8910_1087_68_204/train/state.json  
  inflating: dataset_8910_1087_68_204/validation/data-00000-of-00001.arrow  
  inflating: dataset_8910_1087_68_204/validation/dataset_info.json  
  inflating: dataset_8910_1087_68_204/validation/state.json  
unzipped_file_d

In [9]:
### Create the dataset

dataset = DatasetDict.load_from_disk(unzipped_file_dir_name)

print(f"dataset: {type(dataset)} {dataset.shape}\n{dataset}")


dataset: <class 'datasets.dataset_dict.DatasetDict'> {'train': (1087, 208), 'validation': (68, 208), 'test': (204, 208)}
DatasetDict({
    train: Dataset({
        features: ['id', 'text', '265', '818', '820', '312', '719', '822', '266', '314', '315', '824', '825', '826', '829', '610', '611', '612', '613', '614', '615', '616', '618', '630', '631', '633', '636', '637', '660', '661', '662', '668', '670', '672', '673', '676', '677', '678', '681', '682', '683', '178', '180', '181', '182', '693', '696', '694', '697', '699', '700', '701', '189', '190', '703', '188', '705', '698', '196', '702', '198', '199', '704', '706', '708', '203', '204', '205', '718', '206', '208', '209', '210', '720', '724', '717', '214', '726', '216', '215', '722', '754', '755', '762', '763', '764', '765', '766', '767', '769', '770', '771', '772', '773', '774', '775', '263', '777', '778', '779', '268', '267', '270', '271', '272', '273', '274', '787', '276', '277', '278', '791', '788', '785', '794', '795', '796', '284',

As we can see, the dataset contains 3 splits: one for training, one for validation and one for testing.

Let's test the first example of the training split:

In [10]:
example = dataset['train'][0]
print(f"example: {type(example)} {example.keys()}\n{example}")

example: <class 'dict'> dict_keys(['id', 'text', '265', '818', '820', '312', '719', '822', '266', '314', '315', '824', '825', '826', '829', '610', '611', '612', '613', '614', '615', '616', '618', '630', '631', '633', '636', '637', '660', '661', '662', '668', '670', '672', '673', '676', '677', '678', '681', '682', '683', '178', '180', '181', '182', '693', '696', '694', '697', '699', '700', '701', '189', '190', '703', '188', '705', '698', '196', '702', '198', '199', '704', '706', '708', '203', '204', '205', '718', '206', '208', '209', '210', '720', '724', '717', '214', '726', '216', '215', '722', '754', '755', '762', '763', '764', '765', '766', '767', '769', '770', '771', '772', '773', '774', '775', '263', '777', '778', '779', '268', '267', '270', '271', '272', '273', '274', '787', '276', '277', '278', '791', '788', '785', '794', '795', '796', '284', '797', '285', '288', '289', '802', '291', '800', '293', '806', '807', '803', '297', '298', '811', '300', '812', '813', '815', '814', '817',

In [11]:
### Create the label list and the id2label and label2id mappings.

"""
labels
    if dataset 7_1000_125_125  , 48 labels
    if dataset 7_128_18_54     , 42 labels
    if dataset 8910_1087_68_204, 206 labels
"""

labels = [label for label in dataset['train'].features.keys() if label not in ['id', 'text']]
labels.sort()
print(f"labels: {type(labels)} {len(labels)}\n{labels}")

id2label = {idx:label for idx, label in enumerate(labels)}
print(f"id2label: {type(id2label)} {len(id2label)}\n{id2label}")

label2id = {label:idx for idx, label in enumerate(labels)}
print(f"label2id: {type(label2id)} {len(label2id)}\n{label2id}")


labels: <class 'list'> 206
['178', '180', '181', '182', '183', '188', '189', '190', '196', '198', '199', '203', '204', '205', '206', '208', '209', '210', '214', '215', '216', '263', '265', '266', '267', '268', '270', '271', '272', '273', '274', '275', '276', '277', '278', '283', '284', '285', '286', '288', '289', '291', '292', '293', '297', '298', '300', '301', '302', '303', '304', '308', '309', '310', '311', '312', '313', '314', '315', '316', '317', '318', '320', '321', '322', '324', '358', '359', '365', '366', '367', '369', '372', '378', '379', '380', '381', '382', '384', '385', '396', '397', '398', '399', '400', '401', '402', '403', '404', '405', '406', '407', '410', '411', '610', '611', '612', '613', '614', '615', '616', '618', '630', '631', '633', '636', '637', '660', '661', '662', '668', '670', '672', '673', '676', '677', '678', '681', '682', '683', '693', '694', '696', '697', '698', '699', '700', '701', '702', '703', '704', '705', '706', '708', '717', '718', '719', '720', '722',

The dataset consists of texts, labeled with one or more skills.

Let's create a list that contains the labels, as well as 2 dictionaries that map labels to integers and back.

In [12]:
### Upload the label list as a JSON file to the HF repo_id_dataset

labels_path = "labels.json"
with open(labels_path, 'w') as f:
    json.dump(labels, f)
print(f"labels saved to {labels_path}")


upload_file(
    path_or_fileobj = labels_path,
    path_in_repo    = labels_path,
    repo_id         = HF_name,
    repo_type       = "dataset"
    )
print(f"labels uploaded to https://huggingface.co/datasets/{HF_name}/tree/main/{labels_path}")



labels saved to labels.json


No files have been modified since last commit. Skipping to prevent empty commit.


labels uploaded to https://huggingface.co/datasets/claudelepere/skill_classification/tree/main/labels.json


## Preprocess data

As models like BERT don't expect text as direct input, but rather **`input_ids`**, etc., we tokenize the text using the tokenizer. Here I'm using the `AutoTokenizer` API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a **matrix of shape (batch_size, num_labels)**. Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' **BCEWithLogitsLoss** (which the model will use) will complain, as explained [here](https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3).

### Preprocess (examples, not example, because batched=True => examples is a batch)

In [13]:
### Tokenize 'text'

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [14]:
def preprocess_data(examples, indices):
  text = examples['text']    # Batch of texts

  encoding = tokenizer(
      text,                             # Tokenize text
      truncation     = True,
      padding        = 'max_length',
      max_length     = 512,
      return_tensors = 'pt'             # Return PyTorch tensors
      )

  # Create an empty label matrix
  labels_matrix = torch.zeros((len(text), len(labels)), dtype=torch.float32)
  #print(f"labels_matrix: {type(labels_matrix)} {labels_matrix.shape}")

  # Populate label matrix
  for idx, label in enumerate(labels):
    #print(f"idx:{idx} label:{label}")
    if label in examples:
      labels_matrix[:, idx] = torch.tensor(
          [1.0 if val else 0.0 for val in examples[label]],
          dtype=torch.float32
          )
  #print(f"labels_matrix: {type(labels_matrix)} {labels_matrix.shape}")

  # Add labels to the encoding
  encoding['labels'] = labels_matrix
  #print(f"encoding['labels']: {encoding['labels']}")

  return encoding


In [15]:
### Create the encoded dataset

encoded_dataset = dataset.map(
    preprocess_data,
    batched        = True,
    remove_columns = dataset['train'].column_names,
    with_indices   = True
    )

Map:   0%|          | 0/1087 [00:00<?, ? examples/s]

Map:   0%|          | 0/68 [00:00<?, ? examples/s]

Map:   0%|          | 0/204 [00:00<?, ? examples/s]

In [16]:
example = encoded_dataset['train'][0]
print(f"example: {type(example)} {example.keys()}\n{example}")

print()

print(f"example['input_ids']: {type(example['input_ids'])} {len(example['input_ids'])}\n{example['input_ids']}")
print(f"example['token_type_ids']: {type(example['token_type_ids'])} {len(example['token_type_ids'])}\n{example['token_type_ids']}")
print(f"example['attention_mask']: {type(example['attention_mask'])} {len(example['attention_mask'])}\n{example['attention_mask']}")
print(f"example['labels']:  {type(example['labels'])} {len(example['labels'])}\n{example['labels']}")


example: <class 'dict'> dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
{'input_ids': [101, 4078, 15530, 2401, 1011, 3737, 2961, 1012, 5658, 1010, 29296, 1010, 5604, 1010, 3231, 1010, 4773, 4078, 15530, 2401, 4054, 2229, 11937, 8376, 3802, 24501, 26029, 3736, 14454, 7616, 14895, 2099, 1037, 2474, 2933, 9031, 3802, 1037, 1048, 1005, 24155, 2139, 1048, 1005, 3947, 3795, 2139, 3231, 1025, 28589, 2099, 1048, 1005, 7781, 4078, 5852, 23758, 13665, 19554, 3488, 2139, 3231, 3802, 8740, 21854, 4859, 16252, 2139, 2139, 24759, 10448, 13665, 10439, 22494, 6961, 1025, 16101, 17071, 1010, 5009, 2099, 3802, 7505, 3726, 10484, 2099, 1048, 1005, 20302, 23274, 1010, 2474, 13120, 1010, 2474, 28616, 2063, 4372, 1107, 2226, 12229, 3802, 1048, 1005, 7781, 4078, 25222, 2139, 3231, 1010, 4078, 8853, 2139, 3231, 3802, 4078, 19796, 2139, 3231, 1012, 19684, 4649, 2832, 2271, 4839, 11390, 10364, 8909, 4765, 18095, 4649, 5884, 2229, 1040, 1005, 2572, 20806, 21223, 3802, 2777, 7913, 4372, 110

In [17]:
tokenizer.decode(example['input_ids'])

"[CLS] desmosia - quality officer. net, sql, testing, test, web desmosia principales taches et responsabilites aider a la planification et a l ' estimation de l ' effort global de test ; superviser l ' execution des tests conformement aux plans de test et au calendrier de deploiement approuves ; diriger, guider et surveiller l ' analyse, la conception, la mise en œuvre et l ' execution des cas de test, des procedures de test et des suites de test. examiner les processus existants pour identifier les domaines d ' amelioration et mettre en œuvre les meilleures pratiques. contribuer a des solutions d ' automatisation des tests robustes, evolutives et fiables pour ameliorer l ' efficacite des tests. contribuer a la planification tactique et strategique des futurs programmes de qualite. inventer des solutions elegantes et eliminer les blocages. rapports reguliers au pmo et au departement qualite. collaborer avec les equipes commerciales et techniques pour resoudre des problemes complexes. t

In [18]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['182', '208', '636']

Finally, we set the format of our data to PyTorch tensors. This will turn the training, validation and test sets into standard PyTorch [datasets](https://pytorch.org/docs/stable/data.html).

In [19]:
encoded_dataset.set_format("torch")    # Ensures correctness and compatibility with PyTorch pipelines

## Define model

Here we define a **model that includes a pre-trained base (i.e. the weights from bert-base-uncased) are loaded, with a random initialized classification head (linear layer) on top**. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

This is also printed by the warning.

We set the `problem_type` to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely [**BCEWithLogitsLoss**](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)). We also make sure the output layer has `len(labels)` output neurons, and we set the id2label and label2id mappings.

In [20]:
### Define the model

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    problem_type="multi_label_classification",
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
    )

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Train the model!

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things:

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

In [21]:
batch_size  = 8
metric_name = "f1"

### TrainingArguments

In [22]:
output_dir = "training_results"  # where model predictions and checkpoints will be written during training
args = TrainingArguments(
    output_dir                  = output_dir,
    overwrite_output_dir        = True,
    logging_dir                 = "logs",
    logging_steps               = 50,
    save_steps                  = 500,
    save_total_limit            = 2,
    eval_strategy               = "epoch",
    save_strategy               = "epoch",
    learning_rate               = 2e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size  = batch_size,
    num_train_epochs            = 5,
    weight_decay                = 0.01,
    load_best_model_at_end      = True,
    metric_for_best_model       = metric_name,
    #push_to_hub                 = True,             # push the model to the HF Hub
    run_name                   = "BERT-multilabel-lr2e5-epochs5-datasetHF_128_18_54",
    report_to                  = "none" #"wandb"
    )

We are also going to compute metrics while training. For this, we need to define a `compute_metrics` function, that returns a dictionary with the desired metric values.

In [23]:
### Metrics

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.2):
    _average = 'micro'    # 'micro' or 'weighted'

    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs   = sigmoid(torch.Tensor(predictions))

    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1

    # finally, compute metrics
    y_true               = labels
    f1                   = f1_score               (y_true=y_true, y_pred=y_pred, average=_average)    #, zero_division=1)
    precision            = precision_score        (y_true=y_true, y_pred=y_pred, average=_average)    #, zero_division=1)
    recall               = recall_score           (y_true=y_true, y_pred=y_pred, average=_average)    #, zero_division=1)
    roc_auc              = roc_auc_score          (y_true=y_true, y_score=probs, average=_average)
    precision_recall_auc = average_precision_score(y_true=y_true, y_score=probs, average=_average)
    accuracy             = accuracy_score         (y_true=y_true, y_pred=y_pred)

    # return as dictionary
    metrics = {
        'f1'                  : f1,
        'precision'           : precision,
        'recall'              : recall,
        'roc_auc'             : roc_auc,
        'precision_recall_auc': precision_recall_auc,
        'accuracy'            : accuracy
        }

    return metrics


In [24]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multi_label_metrics(
        predictions = preds,
        labels      = p.label_ids
        )
    return result


Let's verify a batch as well as a forward pass:

In [25]:
print(f"inputids:        {type(encoded_dataset['train']['input_ids'][0])}      {encoded_dataset['train']['input_ids'][0].shape}")
print(f"token_type_ids': {type(encoded_dataset['train']['token_type_ids'][0])} {encoded_dataset['train']['token_type_ids'][0].shape}")
print(f"attention_mask:  {type(encoded_dataset['train']['attention_mask'][0])} {encoded_dataset['train']['attention_mask'][0].shape}")
print(f"labels:          {type(encoded_dataset['train'][0]['labels'])}         {encoded_dataset['train'][0]['labels'].shape}")

inputids:        <class 'torch.Tensor'>      torch.Size([512])
token_type_ids': <class 'torch.Tensor'> torch.Size([512])
attention_mask:  <class 'torch.Tensor'> torch.Size([512])
labels:          <class 'torch.Tensor'>         torch.Size([206])


In [26]:
### Execute the forward pass

outputs = model(
    input_ids      = encoded_dataset['train']['input_ids'][0].unsqueeze(0),
    attention_mask = encoded_dataset['train']['attention_mask'][0].unsqueeze(0),
    labels         = encoded_dataset['train'][0]['labels'].unsqueeze(0)
    )

print(f"outputs: {type(outputs)} {outputs.keys()}\n{outputs}")


outputs: <class 'transformers.modeling_outputs.SequenceClassifierOutput'> odict_keys(['loss', 'logits'])
SequenceClassifierOutput(loss=tensor(0.7156, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-2.0777e-01,  4.8046e-01,  4.3335e-02,  1.4716e-01,  5.8410e-01,
          2.4881e-01, -1.2890e-01, -2.6154e-01, -6.0586e-02,  2.5208e-01,
         -5.2129e-01, -4.7445e-01,  3.1237e-01,  4.2213e-01, -8.1946e-02,
         -3.5199e-01,  4.1424e-01,  1.3597e-01, -6.0092e-01, -7.7877e-01,
          2.9667e-01,  4.9385e-02, -2.5015e-01,  5.1417e-01, -2.4235e-01,
          1.9000e-01,  3.5877e-01,  2.5488e-01,  2.8586e-01,  3.2419e-01,
          1.3825e-01,  2.5022e-01, -3.7708e-01,  3.4707e-02, -3.6947e-02,
          7.5136e-02, -5.9680e-01, -2.9298e-04, -2.5920e-01, -3.5265e-01,
         -8.5790e-02,  3.6636e-01,  3.8846e-01,  2.0418e-01, -1.7808e-02,
          1.4121e-02,  1.8668e-01,  2.7122e-01,  7.5491e-01,  6.2560e-01,
         -1.7345e-01,  5.2444e-01,  2.5910e-01,  3.47

Let's start training!

In [27]:
### Create the trainer

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
    )


<ipython-input-27-72d5597b5b4d>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [28]:
### Train, save the results as a JSON file

train_output  = trainer.train()

train_results = {
    'global_step':   train_output.global_step,    # total steps completed during training
    'training_loss': train_output.training_loss,  # average loss during training
    'metrics':       train_output.metrics         # dictionary of metrics
}

with open("train_results.json", "w") as f:
  json.dump(train_results, f, indent=4)


Epoch,Training Loss,Validation Loss,F1,Precision,Recall,Roc Auc,Precision Recall Auc,Accuracy
1,0.344400,0.211167,0.064438,0.034995,0.406130,0.631491,0.044257,0.000000
2,0.148300,0.126796,0.146096,0.213235,0.111111,0.737691,0.097222,0.000000
3,0.111400,0.105057,0.146096,0.213235,0.111111,0.776970,0.113751,0.000000
4,0.102800,0.097358,0.146096,0.213235,0.111111,0.797541,0.119393,0.000000
5,0.097100,0.095270,0.146096,0.213235,0.111111,0.802284,0.122443,0.000000


## Evaluate

After training, we evaluate our model on the validation set.

In [29]:
### Evaluate, save the output as a JSON file

eval_output  = trainer.evaluate()

with open("eval_results.json", "w") as f:
  json.dump(eval_output, f, indent=4)


In [30]:
### Save the model

model_path = "model"
trainer.save_model(model_path)


In [31]:
### Upload the model and the tokenizer to the HF repo_id_model

model     = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

model.push_to_hub(repo_id_model)
tokenizer.push_to_hub(repo_id_model)


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/claudelepere/skill_classification/commit/1e16b6a5bc046e858c323e03823b5c0b9e5c104a', commit_message='Upload tokenizer', commit_description='', oid='1e16b6a5bc046e858c323e03823b5c0b9e5c104a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/claudelepere/skill_classification', endpoint='https://huggingface.co', repo_type='model', repo_id='claudelepere/skill_classification'), pr_revision=None, pr_num=None)

In [32]:
### Upload train_results.json and eval_results.json to the HF repo_id_dataset

# BETTER to upload to wanddb repo?

upload_file(
    path_or_fileobj = "train_results.json",
    path_in_repo    = "train_results.json",
    repo_id         = HF_name,
    repo_type       = "dataset"
    )

upload_file(
    path_or_fileobj = "eval_results.json",
    path_in_repo    = "eval_results.json",
    repo_id         = HF_name,
    repo_type       = "dataset"
    )


CommitInfo(commit_url='https://huggingface.co/datasets/claudelepere/skill_classification/commit/43236ce6226676a6ab6fb4a2a96f0f93e0feb58e', commit_message='Upload eval_results.json with huggingface_hub', commit_description='', oid='43236ce6226676a6ab6fb4a2a96f0f93e0feb58e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/claudelepere/skill_classification', endpoint='https://huggingface.co', repo_type='dataset', repo_id='claudelepere/skill_classification'), pr_revision=None, pr_num=None)

In [33]:
print("It's the end of the notebook!")

It's the end of the notebook!
